In [1]:
import os

%load_ext autoreload
%autoreload 2

""""
Devo code for moment by moment decoding --> i.e., for each time bin, find strength of representation of
each class of a variable (e.g., shape), as opposed to decoding average accuracy across trials.

Psychoprim --> goal is to decoder the base prims, given an ambiguous middle prim

"""

'"\nDevo code for moment by moment decoding --> i.e., for each time bin, find strength of representation of\neach class of a variable (e.g., shape), as opposed to decoding average accuracy across trials.\n\nPsychoprim --> goal is to decoder the base prims, given an ambiguous middle prim\n\n'

# Load a dataset

To load and plot a dataset of neural activity across population, in a PopAnal class object.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:
# Method 1 - load a single DFallPA
DFallpa = load_handsaved_wrapper()


In [ ]:
# Method 1 - load a single DFallPA
animal = "Diego"
# date = 240523
date = 240515

DFallpa = load_handsaved_wrapper(animal=animal, date=date, version="trial")


In [ ]:
# # Method 2 - Combine two dfallpa
# DFallpa1 = load_handsaved_wrapper(animal="Diego", date=230630, version="trial")
# DFallpa2 = load_handsaved_wrapper(animal="Diego", date=230630, version="stroke")
# DFallpa = pd.concat([DFallpa1, DFallpa2]).reset_index(drop=True)


In [ ]:
# # Method 2 - Combine two dfallpa
# animal = "Diego"
# date = 231211
# COMBINE_AREAS = True

# DFallpa1 = load_handsaved_wrapper(animal=animal, date=date, version="trial", combine_areas=COMBINE_AREAS, use_time=False)
# DFallpa2 = load_handsaved_wrapper(animal=animal, date=date, version="stroke", combine_areas=COMBINE_AREAS, use_time=False)
# DFallpa = pd.concat([DFallpa1, DFallpa2]).reset_index(drop=True)


# Preprocessing

In [ ]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:

# Prune to chans that are common across pa for each bregion (intersection of chans)|
dfpa_match_chans_across_pa_each_bregion(DFallpa)

In [ ]:
# SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/{animal}-{date}"
SAVEDIR = f"/tmp/PSYCHO_SP/{animal}-{date}"
os.makedirs(SAVEDIR, exist_ok=True)
print(SAVEDIR)

# Preprocessing

##### Devo -- removing noisy channels

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concatbregion_preprocess_clean_bad_channels
dfpa_concatbregion_preprocess_clean_bad_channels(DFallpa, PLOT=False)

# Sqrt normalization

In [ ]:
for pa in DFallpa["pa"]:
    pa.X = pa.X**0.5

### FR Normalization

##### Method 1 - each PA normalize independently

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
plot_savedir = "/tmp"
dfallpa_preprocess_fr_normalization(DFallpa, fr_normalization_method, plot_savedir)

##### Method 2 - Concat events (for each bregion) and normalize all same way


In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
dfpa_concat_normalize_fr_split_multbregion(DFallpa)


##### Method 3 - concat events (flexible version, only constrianed to have same n chans across PA) [works if have both trial and stroke!]

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion_flex
fr_mean_subtract_method = "across_time_bins"
# fr_mean_subtract_method = "each_time_bin"
PLOT=False

pa = DFallpa["pa"].values[10]
pa.plotNeurHeat(0)

dfpa_concat_normalize_fr_split_multbregion_flex(DFallpa, fr_mean_subtract_method, PLOT)

pa = DFallpa["pa"].values[10]
pa.plotNeurHeat(0)

# Extract relevant variables


In [ ]:
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import dfallpa_preprocess_condition
# shape_var_suff = "shapesemgrp"|
# loc_var_suff = "loc_on_clust"
shape_var_suff = "shape"
loc_var_suff = "loc"
dfallpa_preprocess_condition(DFallpa, shape_var_suff, loc_var_suff)

# Extract beh data

In [ ]:
from neuralmonkey.analyses.decode_moment import analy_psychoprim_prepare_beh_dataset
DSmorphsets, map_tc_to_morph_info, map_morphset_to_basemorphinfo, map_tcmorphset_to_idxmorph, map_tcmorphset_to_info, map_morphsetidx_to_assignedbase_or_ambig, map_tc_to_morph_status = analy_psychoprim_prepare_beh_dataset(animal, date)

In [ ]:
# Prune neural data to keep only those trialcodes that are in DSmorphsets
list_trialcode = DSmorphsets.Dat["trialcode"].unique().tolist()
DFallpa["pa"] = [pa.slice_by_labels("trials", "trialcode", list_trialcode) for pa in DFallpa["pa"].values]


# Train / test

In [ ]:
from neuralmonkey.analyses.decode_moment import train_decoder_helper, pipeline_train_test_scalar_score, pipeline_train_test_scalar_score_with_splits, pipeline_train_test_scalar_score_mult_train_dataset, test_decoder_helper, train_decoder_helper_extract_train_dataset
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import get_dataset_params
import os

In [ ]:
# get list of base prim names

### [GOOD] Train on data for a specific morphset -- get average score across trials


In [ ]:
from pythonlib.tools.plottools import savefig
# SAVEDIR = "/tmp/decoder_each_morphset"
print(SAVEDIR)

##### [GOOD] Loop over all morphsets

In [ ]:
from neuralmonkey.analyses.decode_moment import analy_psychoprim_score_postsamp
analy_psychoprim_score_postsamp(DFallpa, DSmorphsets, 
                                    map_tcmorphset_to_idxmorph, map_morphsetidx_to_assignedbase_or_ambig,
                                    map_tcmorphset_to_info,
                                    SAVEDIR, animal, date)

##### METHOD 2 -- new - doing train/test split -- ask if base1/base2 decoders are better than own decoder (for ambig tasks)

In [ ]:
from neuralmonkey.analyses.decode_moment import analy_psychoprim_score_postsamp_better, _analy_psychoprim_score_postsamp_plot_scores


SAVEDIR_BASE = "/tmp/PSYCHOPRIM"
analy_psychoprim_score_postsamp_better(DFallpa, DSmorphsets, 
                                    map_tcmorphset_to_idxmorph, map_morphsetidx_to_assignedbase_or_ambig,
                                    map_tcmorphset_to_info,
                                    SAVEDIR_BASE, PLOT_EACH_IDX=False)


##### METHOD 3 - like method 2 (i..e, using train/test split) but doing one analysis for each morphset (as oposed to each morph task)

In [ ]:
from neuralmonkey.analyses.decode_moment import _analy_psychoprim_score_postsamp_plot_scores, analy_psychoprim_dfscores_condition


In [ ]:
SAVEDIR_BASE = "/tmp/PSYCHONEW"
import seaborn as sns
from pythonlib.tools.pandastools import savefig
from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap


In [ ]:
bregion = "PMv"
list_bregion = [bregion]

# train_on_which_prims = "all" # all within morpjset
# train_on_which_prims = "base" # just the 2 base prims



In [ ]:
version = 1
from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap


In [ ]:
do_agg_of_user_test_data

In [ ]:
dfscores_both

In [ ]:
savedir = f"{SAVEDIR}/{bregion}/morphset={morphset}/plots"
os.makedirs(savedir, exist_ok=True)
print("Saving plots at... ", savedir)
_analy_psychoprim_score_postsamp_plot_scores(dfscores, savedir)


In [ ]:
# 1. restrucict to idx_within that have a given set of decoders.
# 2. 
from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars

# Clean comparison
levels_get = ["base1", "same", "base2"]
lenient_allow_data_if_has_n_levels = 2
# lenient_allow_data_if_has_n_levels = len(levels_get)

n_min = 1
dfscores_this, dict_dfthis = extract_with_levels_of_conjunction_vars(DFSCORES, "decoder_class_semantic_good", 
                                                             ["morph_set_idx|idx_within"], levels_get, n_min,  
                                                             lenient_allow_data_if_has_n_levels=lenient_allow_data_if_has_n_levels,
                                                             prune_levels_with_low_n=True, 
                                                             plot_counts_heatmap_savepath="/tmp/counts.png")

from pythonlib.tools.pandastools import aggregGeneral
dfscores_this_agg = aggregGeneral(dfscores_this, ["morph_set_idx|idx_within", "pa_class", "decoder_class"], ["score"], nonnumercols="all")

_analy_psychoprim_score_postsamp_plot_scores(dfscores_this_agg, savedir)


In [ ]:
dfscores_this

In [ ]:
dfscores.columns

In [ ]:

# # Plot 
# var_score = "score"
# x_var = "decoder_class_semantic_good"
# x_order = ("base1", "interm1", "same", "interm2", "base2")
# col = "trial_morph_assigned_to_which_base"
# fig = sns.catplot(data=dfscores_this, x=x_var, y=var_score, hue = "morph_set_idx", kind="point", errorbar=("ci", 68), 
#                 col=col, order=x_order)
# for ax in fig.axes.flatten():
#     ax.axhline(0, color="k", alpha=0.3)
# savefig(fig, f"{savedir}/decoder_class-x_var={x_var}-var_score={var_score}-col={col}-4.pdf")

# fig = sns.catplot(data=dfscores_this, x=x_var, y=var_score, hue = "morph_set_idx|idx_within", kind="point", errorbar=("ci", 68), 
#                 col=col, order=x_order)
# for ax in fig.axes.flatten():
#     ax.axhline(0, color="k", alpha=0.3)
# savefig(fig, f"{savedir}/decoder_class-x_var={x_var}-var_score={var_score}-col={col}-4b.pdf")

##### Q: is the intermediate lower than same and base?
# levels_get = ["base1", "interm1", "same"]
# levels_get = ["base2", "interm2", "same"]
# levels_get = ["base2", "interm2", "same"]
levels_get = ["same", "intermthis", "basethis"]

# Clean comparison
lenient_allow_data_if_has_n_levels = len(levels_get)

n_min = 1
dfscores_this, dict_dfthis = extract_with_levels_of_conjunction_vars(DFSCORES, "recoded_decoder", 
                                                             ["morph_set_idx|idx_within"], levels_get, n_min,  
                                                             lenient_allow_data_if_has_n_levels=lenient_allow_data_if_has_n_levels,
                                                             prune_levels_with_low_n=True, 
                                                             plot_counts_heatmap_savepath="/tmp/counts.png")

from pythonlib.tools.pandastools import aggregGeneral
# dfscores_this_agg = aggregGeneral(dfscores_this, ["morph_set_idx|idx_within", "pa_class", "decoder_class", "trial_morph_assigned_to_which_base"], ["score"], nonnumercols="all")

_analy_psychoprim_score_postsamp_plot_scores(dfscores_this_agg, savedir)



In [ ]:

# Plot 
# 1. restrucict to idx_within that have a given set of decoders.
# 2. 
from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars

##### Q: 
# match the morphset to ask whether ambig is diff from not-ambig
levels_get = ["not_ambig_base1", "ambig_base1"]
n_min = 1
dfscores_this, dict_dfthis = extract_with_levels_of_conjunction_vars(DFSCORES, "trial_morph_assigned_to_which_base", 
                                                             ["morph_set_idx", "decoder_class_semantic_good"], levels_get, n_min,  
                                                             lenient_allow_data_if_has_n_levels=len(levels_get), 
                                                             prune_levels_with_low_n=True, 
                                                             plot_counts_heatmap_savepath="/tmp/counts.png")

from pythonlib.tools.pandastools import aggregGeneral
dfscores_this_agg = aggregGeneral(dfscores_this, ["morph_set_idx|idx_within", "pa_class", "decoder_class"], ["score"], nonnumercols="all")

_analy_psychoprim_score_postsamp_plot_scores(dfscores_this_agg, savedir)

# # Plot 
# var_score = "score"
# x_var = "decoder_class_semantic_good"
# x_order = ("base1", "interm1", "same", "interm2", "base2")
# col = "trial_morph_assigned_to_which_base"
# fig = sns.catplot(data=dfscores_this, x=x_var, y=var_score, hue = "morph_set_idx", kind="point", errorbar=("ci", 68), 
#                 col=col, order=x_order)
# for ax in fig.axes.flatten():
#     ax.axhline(0, color="k", alpha=0.3)
# savefig(fig, f"{savedir}/decoder_class-x_var={x_var}-var_score={var_score}-col={col}-4.pdf")

# fig = sns.catplot(data=dfscores_this, x=x_var, y=var_score, hue = "morph_set_idx|idx_within", kind="point", errorbar=("ci", 68), 
#                 col=col, order=x_order)
# for ax in fig.axes.flatten():
#     ax.axhline(0, color="k", alpha=0.3)
# savefig(fig, f"{savedir}/decoder_class-x_var={x_var}-var_score={var_score}-col={col}-4b.pdf")


In [ ]:
sdir = "/tmp/test"
os.makedirs(sdir, exist_ok=True)
Dc.scalar_score_df_plot_summary(dfscores_testsplit, sdir)

In [ ]:
from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
dfthis = dfscores_testsplit
var_score = "score"
task_vars = ["morph_set_idx", "pa_class"]

_, fig = plot_45scatter_means_flexible_grouping(dfthis, "decoder_class_semantic_good", "base1", "base2",
                                    "trial_morph_assigned_to_which_base", var_score, task_vars,
                                    plot_text=True, shareaxes=True, SIZE=6, alpha=0.3);

_, fig = plot_45scatter_means_flexible_grouping(dfthis, "trial_morph_assigned_to_which_base", "ambig_base1", "ambig_base2",
                                    "decoder_class", var_score, task_vars,
                                    plot_text=True, shareaxes=True, SIZE=6, alpha=0.3);
# savefig(fig, f"{savedir}/scatter-x=decoder_class_semantic_good-y=trial_morph_assigned_to_which_base-dfthis_str={dfthis_str}-var_score={var_score}.pdf")


# MULTIPLE AREAS - combined analysis, loading all saved data

In [ ]:
import pandas as pd
from neuralmonkey.classes.session import _REGIONS_IN_ORDER, _REGIONS_IN_ORDER_COMBINED
from pythonlib.tools.pandastools import grouping_plot_n_samples_conjunction_heatmap, stringify_values, aggregGeneral
from neuralmonkey.scripts.analy_decode_moment_psychometric import prune_dfscores_good_morphset


In [ ]:
def _plot(dfscores, huevar, xvar, xvalues, huevalues, savedir):
    from pythonlib.tools.pandastools import plot_subplots_heatmap
    import seaborn as sns
    from pythonlib.tools.snstools import rotateLabel
    from pythonlib.tools.pandastools import savefig

    for var_score, diverge in [("score", False), ("score_adjusted", True)]:
        fig, _ = plot_subplots_heatmap(dfscores, huevar, xvar, 
                            var_score, "bregion", share_zlim=True, diverge=diverge,
                            row_values=huevalues, col_values=xvalues)
        savefig(fig, f"{savedir}/heatmap-varscore={var_score}.pdf")
    plt.close("all")

    # Plot as point plot
    for kind in ["point", "bar"]:
        for var_score in ["score", "score_adjusted"]:

            fig = sns.catplot(data=dfscores, x=xvar, y=var_score, hue=huevar, 
                        col="bregion", col_wrap=6, kind=kind, errorbar=("ci", 68),
                        order = xvalues)
            if var_score == "score":
                yzero = 0.5
            else:
                yzero = 0.
            for ax in fig.axes.flatten():
                ax.axhline(yzero)            
            rotateLabel(fig)
            savefig(fig, f"{savedir}/catplot-varscore={var_score}-kind={kind}.pdf")
    plt.close("all")

    kind = "point"
    for var_score in ["score", "score_adjusted"]:

        fig = sns.catplot(data=dfscores, x=xvar, y=var_score, hue="morph_set_idx", 
                            row=huevar, col="bregion", kind=kind, errorbar=("ci", 68), order = xvalues)
        if var_score == "score":
            yzero = 0.5
        else:
            yzero = 0.
        for ax in fig.axes.flatten():
            ax.axhline(yzero)            
        rotateLabel(fig)
        savefig(fig, f"{savedir}/catplot-varscore={var_score}-kind={kind}-splitby-morph_set_idx.pdf")
    plt.close("all")



In [17]:
SAVEDIR_BASE = "/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP"

combine = True
downsample_trials = False
version = 1

WHICH_PLOTS = [1]
for animal, date in [("Diego", 240517), ("Diego", 240515), ("Pancho", 240521), ("Pancho", 240524),  ("Diego", 240523), ("Diego", 240521),  ]:
    for TWIND_TEST in [(0.05, 1.2)]:

        if combine:
            list_bregion = _REGIONS_IN_ORDER_COMBINED
        else:
            list_bregion = _REGIONS_IN_ORDER

        list_dfscores = []
        for bregion in list_bregion:
            path = f"{SAVEDIR_BASE}/{animal}-{date}-logistic-combine={combine}/downsample_trials={downsample_trials}-TWIND_TEST={TWIND_TEST}-version={version}/{bregion}/DFSCORES.pkl"
            print("Loading ... ", path)
            dfscores = pd.read_pickle(path)
            dfscores["bregion"] = bregion
            list_dfscores.append(dfscores)
        DFSCORES = pd.concat(list_dfscores).reset_index(drop=True)

        SAVEDIR = f"{SAVEDIR_BASE}/MULT_BREGION/{animal}-{date}-logistic-combine={combine}/downsample_trials={downsample_trials}-TWIND_TEST={TWIND_TEST}-version={version}"
        print("SAving at: ", SAVEDIR)

        ### PReprocessing
        assert len(DFSCORES["twind"].unique())==1, "code below assumes"

        # Get score relative to chance (50%)
        score_chance = 0.5
        DFSCORES["score_adjusted"] = (DFSCORES["score"] - score_chance)/(1. - score_chance)

        ######################## PLOTS
        if 0 in WHICH_PLOTS:
            ### (1) Biomdal, plots, categorization.
            plot_bimodal_even_if_not_switch(DFSCORES, SAVEDIR)

        if 1 in WHICH_PLOTS:
            ### (2) Ambig trials, interanl state? (scalar plots)
            for do_agg_over_trials in [False, True]:
                for morphset_get in [None, "good_ones"]:

                    dfscores = prune_dfscores_good_morphset(DFSCORES, morphset_get, animal, date)

                    if do_agg_over_trials:
                        dfscores = aggregGeneral(dfscores, 
                                                ["bregion", "morph_set_idx|idx_within", "idx_within|assigned", "pa_class", "decoder_class", "twind"], 
                                                ["score", "score_adjusted"], 
                                                nonnumercols=["trial_morph_assigned_to_which_base"])
                    # (1) 
                    huevar = "decoder_class_semantic_good"
                    xvar = "trial_morph_assigned_to_which_base"
                    xvalues = ["base1", "not_ambig_base1", "ambig_base1", "ambig_base2", "not_ambig_base2", "base2"]
                    huevalues = ["base1", "base2"]

                    savedir = f"{SAVEDIR}/do_agg_over_trials={do_agg_over_trials}-morphset_get={morphset_get}/xvar={xvar}"
                    os.makedirs(savedir, exist_ok=True)

                    _plot(dfscores, huevar, xvar, xvalues, huevalues, savedir)

                    # (2)
                    huevar = "recoded_decoder"
                    xvar = "recoded_trial_morph"
                    xvalues = ["ambig", "not_ambig", "base"]
                    huevalues = ["baseother", "basethis"]

                    savedir = f"{SAVEDIR}/do_agg_over_trials={do_agg_over_trials}-morphset_get={morphset_get}/xvar={xvar}"
                    os.makedirs(savedir, exist_ok=True)

                    _plot(dfscores, huevar, xvar, xvalues, huevalues, savedir)

        if 2 in WHICH_PLOTS:
            ### (1) "Logistic vs. linear" plots, categorization.
            plot_logistic_vs_linear_curve(DFSCORES, SAVEDIR)



Loading ...  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240517-logistic-combine=True/downsample_trials=False-TWIND_TEST=(0.05, 1.2)-version=1/M1/DFSCORES.pkl
Loading ...  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240517-logistic-combine=True/downsample_trials=False-TWIND_TEST=(0.05, 1.2)-version=1/PMv/DFSCORES.pkl
Loading ...  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240517-logistic-combine=True/downsample_trials=False-TWIND_TEST=(0.05, 1.2)-version=1/PMd/DFSCORES.pkl
Loading ...  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240517-logistic-combine=True/downsample_trials=False-TWIND_TEST=(0.05, 1.2)-version=1/dlPFC/DFSCORES.pkl
Loading ...  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240517-logistic-combine=True/downsample_trials=False-TWIND_TEST=(0.05, 1.2)-version=1/vlPFC/DFSCORES.pkl
Loading ...  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO

NameError: name 'prune_dfscores_good_morphset' is not defined

### Categorization, for non-ambiguous images (bimodal across trials)

In [ ]:
def plot_bimodal_even_if_not_switch(DFSCORES, SAVEDIR):
    """
    Question, qhether there is bimodal distribtuion of scores, even for inidices that are not switchign in beh
    """
    from pythonlib.tools.snstools import rotateLabel
    from pythonlib.tools.plottools import savefig
    import seaborn as sns
    import os
    import numpy as np
    from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping


    def F(trial_morph_assigned_to_which_base):
        a = "base1" in trial_morph_assigned_to_which_base
        b = "base2" in trial_morph_assigned_to_which_base
        if a and not b:
            return "base1"
        elif not a and b:
            return "base2"
        else:
            print(trial_morph_assigned_to_which_base)
            assert False

    DFSCORES["trial_morph_assigned_to_which_base_clean"] = [F(x) for x in DFSCORES["trial_morph_assigned_to_which_base"]]


    DFSCORES["score_logodds"] = np.log(DFSCORES["score"]/(1 - DFSCORES["score"]))
    assert len(DFSCORES["twind"].unique())==1
    list_bregion = DFSCORES["bregion"].unique().tolist()
    list_morphset = DFSCORES["morph_set_idx"].unique().tolist()
    for bregion in list_bregion:
        for morphset in list_morphset:

            # savedir = f"{SAVEDIR}/logistic_categorical_plots/bregion={bregion}-morphset={morphset}"
            savedir = f"{SAVEDIR}/plot_bimodal_even_if_not_switch/bregion={bregion}-morphset={morphset}"
            os.makedirs(savedir, exist_ok=True)
            print("... Saving figures at: ", savedir)

            dfscores = DFSCORES[(DFSCORES["morph_set_idx"] == morphset) & (DFSCORES["bregion"] == bregion)].reset_index(drop=True)

            for var_score in ["score", "score_logodds"]:

                if False:
                    plot_45scatter_means_flexible_grouping(dfscores, "decoder_class", 0, 99, "idx_within|assigned", var_score, 
                                                        "trialcode", False, shareaxes=True, alpha=0.2)
                    plot_45scatter_means_flexible_grouping(dfscores, "decoder_class", 0, 99, "pa_class", var_score, 
                                                        "trialcode", False, shareaxes=True, alpha=0.2)
                    fig = sns.displot(data=dfscores, x=var_score, row="decoder_class", col="pa_class", bins=25)
                    for ax in fig.axes.flatten():
                        ax.axvline(0.5, color="k", alpha=0.5)
                    fig = sns.displot(data=dfscores, x=var_score, row="decoder_class", col="pa_class", kind="kde")
                    for ax in fig.axes.flatten():
                        ax.axvline(0.5, color="k", alpha=0.5)

                ##### Quantifying bimodality.
                # (1) Plot all, overlaid.
                for xvar in ["pa_class", "idx_within|assigned"]:
                    order = sorted(dfscores[xvar].unique())

                    fig = sns.catplot(data=dfscores, x=xvar, y=var_score, col="decoder_class", alpha=0.5, 
                                    jitter=True, hue="trial_morph_assigned_to_which_base_clean", order=order)
                    rotateLabel(fig)
                    savefig(fig, f"{savedir}/catplot-x={xvar}-varscore={var_score}-1.pdf")

                    fig = sns.catplot(data=dfscores, x=xvar, y=var_score, col="decoder_class", kind="violin", order=order)
                    rotateLabel(fig)
                    savefig(fig, f"{savedir}/catplot-x={xvar}-varscore={var_score}-2.pdf")

                    fig = sns.catplot(data=dfscores, x=xvar, y=var_score, col="decoder_class", kind="point", order=order)
                    rotateLabel(fig)
                    savefig(fig, f"{savedir}/catplot-x={xvar}-varscore={var_score}-3.pdf")

                    plt.close("all")


In [ ]:
# from pythonlib.tools.statstools import curve_fit_and_test

# def sigmoid(x, L, k, x0, b):
#     return L * (1 / (1 + np.exp(-k*(x-x0)))) + b

# bounds_sigmoid=([0., 0., -6., -5.], [7., 15., 6., 5])

# def linear(x, a, b):
#     return (a*x) + b

# bounds_linear = ([-2, -5], [2, 5])



In [ ]:
# scores = dfthis[var_score].values
# indexes = dfthis["pa_class"].values
# indexes_rank = rank_items(indexes, method="dense")

def compute_score_diffs(dfscores, var_score):
    # var_score = "score_logodds"
    scores_0 = dfscores[dfscores["pa_class"] == 0][var_score]
    scores_1 = dfscores[dfscores["pa_class"] == 1][var_score]
    scores_2 = dfscores[dfscores["pa_class"] == sorted(dfscores["pa_class"].unique())[-2]][var_score]
    scores_3 = dfscores[dfscores["pa_class"] == 99][var_score]

    scorediff_start = np.mean(scores_1) - np.mean(scores_0)
    scorediff_end = np.mean(scores_3) - np.mean(scores_2)
    scorediff_edge = np.mean([scorediff_start, scorediff_end])

    scorediff_total = np.mean(scores_3) - np.mean(scores_0)

    return scorediff_total, scorediff_edge


In [ ]:
def plot_logistic_vs_linear_curve(DFSCORES, SAVEDIR):
    """
    QUestion: whether curve is better fit by logistic vs. lieanr.

    """
    import numpy as np
    from pythonlib.tools.listtools import rank_items
    from pythonlib.tools.statstools import balanced_stratified_kfold
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from pythonlib.tools.plottools import savefig

    # Fit logistic function
    DFSCORES["score_logodds"] = np.log(DFSCORES["score"]/(1 - DFSCORES["score"]))

    assert len(DFSCORES["twind"].unique())==1
    list_bregion = DFSCORES["bregion"].unique().tolist()
    list_morphset = DFSCORES["morph_set_idx"].unique().tolist()
    nfolds = 10
    n_min = 10

    # list_bregion = ["PMv"]
    # list_morphset = [0]
    var_score = "score_logodds"
    var_score_diffs = "score"
    RES = []
    RES_SCOREDIFFS = []
    for bregion in list_bregion:
        for morphset in list_morphset:

            dfscores = DFSCORES[
                (DFSCORES["morph_set_idx"] == morphset) & 
                (DFSCORES["bregion"] == bregion) & 
                (DFSCORES["decoder_class"] == 99)
                ].reset_index(drop=True)

            from pythonlib.tools.pandastools import extract_with_levels_of_var_good
            dfthis, inds_keep = extract_with_levels_of_var_good(dfscores, ["pa_class"], n_min)
            print("Sample size pruning... ", len(dfscores), " --> ", len(dfthis))

            scores = dfthis[var_score].values
            indexes = dfthis["pa_class"].values
            indexes_rank = rank_items(indexes, method="dense")

            ############### SCORE DIFFS (EDGE VS. TOTAL)
            scorediff_total, scorediff_end = compute_score_diffs(dfscores, var_score_diffs)

            for sd, sdv in [
                (scorediff_total, "total"),
                (scorediff_end, "end")
                ]:
                RES_SCOREDIFFS.append({
                    "bregion":bregion,
                    "morphset":morphset,
                    "scorediff":sd,
                    "scorediff_ver":sdv,
                    "var_score":var_score_diffs,
                })
                
            ############### REGRESSION (LOGISTIC VS LINEAR)
            if False:
                smin = scores.min()
                smax = scores.max()
                scores_norm_01 = (scores - smin)/(smax-smin)

        
            # Split into train-test splits
            X = indexes_rank.astype(float) - np.mean(indexes_rank.astype(float))
            Y = scores.astype(float) - np.mean(scores.astype(float))

            


            assert False, "replace this part with curve_fit_and_test_wrapper_compare_logistic_linear"
            folds = balanced_stratified_kfold(None, indexes_rank, nfolds)
            plot_sanity = False

            res = []
            for inds_train, inds_test in folds:
                x_train = X[inds_train]
                y_train = Y[inds_train]
                x_test = X[inds_test]
                y_test = Y[inds_test]

                assert x_test.shape == y_test.shape
                if plot_sanity:
                    fig, axes = plt.subplots(2,2, figsize=(10, 10), sharex=True, sharey=True)
                    ax = axes.flatten()[0]
                    ax.set_title("all data")
                    ax.plot(x, y, "ob", alpha=0.5)
                    # ax.plot(x, sigmoid(x,  2, 1, 0, -0.9), 'go')

                    ax = axes.flatten()[1]
                    ax.set_title("train data")
                    ax.plot(x_train, y_train, "ob", alpha=0.5)
                    
                    ax = axes.flatten()[2]
                    ax.set_title("test data")
                    ax.plot(x_test, y_test, "ob", alpha=0.5)
                    
                    assert False


                y_pred, R2, residuals_pred, residuals_mean = curve_fit_and_test(sigmoid, bounds_sigmoid, x_train, y_train, x_test, y_test, doplot=False)
                res.append({
                    "model":"sigmoid",
                    "y_pred":y_pred,
                    "y_test":y_test,
                    "R2":R2,
                    "residuals_pred":residuals_pred,
                    "residuals_mean":residuals_mean,
                })
                
                y_pred, R2, residuals_pred, residuals_mean = curve_fit_and_test(linear, bounds_linear, x_train, y_train, x_test, y_test, doplot=False)
                res.append({
                    "model":"linear",
                    "y_pred":y_pred,
                    "y_test":y_test,
                    "R2":R2,
                    "residuals_pred":residuals_pred,
                    "residuals_mean":residuals_mean,
                })

            # Collect single result for each model
            dfres = pd.DataFrame(res)




            for model in ["linear", "sigmoid"]:
                # Collect all predicted values, and compute R2
                
                dfresthis = dfres[dfres["model"] == model]
                
                Y_PRED = np.concatenate(dfresthis["y_pred"].tolist())
                Y_TEST = np.concatenate(dfresthis["y_test"].tolist())
                assert Y_PRED.shape == Y_TEST.shape == Y.shape
                from pythonlib.tools.statstools import coeff_determination_R2
                R2, residuals_pred, residuals_mean = coeff_determination_R2(Y_TEST, Y_PRED, False)

                R2_mean = np.mean(dfresthis["R2"])

                print(bregion, " -- ", morphset, " -- ", model, " -- ", R2)

                # Collect output
                RES.append({
                    "model":model,
                    "bregion":bregion,
                    "morphset":morphset,
                    "R2":R2,
                    "R2_mean":R2_mean
                })



    DFRES = pd.DataFrame(RES)
    DFRES_SCOREDIFFS = pd.DataFrame(RES_SCOREDIFFS)

    ###### PLOTS
    savedir = f"{SAVEDIR}/logistic_vs_linear"
    os.makedirs(savedir, exist_ok=True)
    print("... Saving figures at: ", savedir)

    from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
    # import seaborn as sns
    # sns.catplot(data=DFRES, x="model", y="R2", col="bregion", kind="point")
    # sns.catplot(data=DFRES, x="model", y="R2_mean", col="bregion", kind="point")
    # sns.catplot(data=DFRES, x="model", y="R2", col="bregion", kind="point")
    for var_score in ["R2", "R2_mean"]:
        _, fig = plot_45scatter_means_flexible_grouping(DFRES, "model", "linear", "sigmoid", "bregion", var_score, "morphset", shareaxes=True, SIZE=3, fontsize=5);
        savefig(fig, f"{savedir}/logistic_vs_linear-scatter-varscore={var_score}.pdf")

    _, fig = plot_45scatter_means_flexible_grouping(DFRES_SCOREDIFFS, "scorediff_ver", "total", "end", "bregion", "scorediff", "morphset", shareaxes=True, SIZE=3, fontsize=5);
    savefig(fig, f"{savedir}/scorediffs-scatter-1.pdf")

    _, fig = plot_45scatter_means_flexible_grouping(DFRES_SCOREDIFFS, "scorediff_ver", "end", "total", "bregion", "scorediff", "morphset", shareaxes=True, SIZE=3, fontsize=5);
    savefig(fig, f"{savedir}/scorediffs-scatter-2.pdf")

    plt.close("all")


In [ ]:
plot_logistic_vs_linear_curve(DFSCORES, "/tmp/test")

# STATE SPACE VISUALIZATION PLOTS

In [ ]:
PA = extract_single_pa(DFallpa, "PMv", None, "trial")


In [ ]:
scalar_or_traj = "scal"
version = "dpca"

savedir = "/tmp"
twind_pca = (0.1, 0.5)
dpca_var = "seqc_0_shape"
tbin_dur = 0.2
tbin_slide = 0.1
PAredu = PA.dataextract_dimred_wrapper(scalar_or_traj, version, savedir, twind_pca, tbin_dur, tbin_slide,
                                       dpca_var=dpca_var)
print(PAredu.X.shape)

In [ ]:
from neuralmonkey.scripts.analy_decode_moment_psychometric import analy_psychoprim_statespace_euclidian
SAVEDIR = "/tmp"
list_morphset = DSmorphsets.Dat["morph_set_idx"].unique().tolist()
analy_psychoprim_statespace_euclidian(DFallpa, SAVEDIR, map_tcmorphset_to_idxmorph, list_morphset)

# [MULT DATES] STATE SPACE VISUALIZATION PLOTS

In [4]:
from pythonlib.dataset.dataset_analy.psychometric_singleprims import params_good_morphsets_no_switching
import numpy as np
from pythonlib.tools.vectools import projection_onto_axis_subspace
import seaborn as sns
import pickle
from pythonlib.tools.plottools import savefig

In [5]:
from neuralmonkey.classes.session import _REGIONS_IN_ORDER_COMBINED


In [7]:
SAVEDIR_BASE = "/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP"


combine = True
version = "dpca"
subtrmean = False
projtwind = (0.1, 1.2)

scalar_or_traj = "scal"
npcs = 8

# scalar_or_traj = "traj"
# npcs = 6

list_bregion = _REGIONS_IN_ORDER_COMBINED
# list_bregion = ["PMv", "PMd"]

DOPLOTS = False

### 
list_dfproj = []

for animal, date in [("Diego", 240515), ("Diego", 240517), ("Diego", 240521), 
                     ("Diego", 240523), ("Pancho", 240524), 
                     ("Diego", 240731), ("Diego", 240801), ("Diego", 240802), ("Pancho", 240521), 
                     ("Pancho", 240801), ("Pancho", 240802)]:
    # animal = "Pancho"
    # date = 240521

    list_morphset = params_good_morphsets_no_switching(animal, date)

    for morphset in list_morphset:
        # morphset = 7

        for bregion in list_bregion:
            # bregion = "PMd"

            SAVEDIR = f"{SAVEDIR_BASE}/{animal}-{date}-logistic-combine={combine}/statespace_euclidian/bregion={bregion}/morphset={morphset}-ver={scalar_or_traj}-{version}-subtrmean={subtrmean}-projtwind={projtwind}-npcs={npcs}"
            

            # Load data
            path = f"{SAVEDIR}/Cldist.pkl"
            print(path)
            with open(path, "rb") as f:
                Cldist = pickle.load(f)

            path = f"{SAVEDIR}/PAredu.pkl"
            print(path)
            with open(path, "rb") as f:
                PAredu = pickle.load(f)

            # Dir for plotting
            savedir = f"{SAVEDIR_BASE}/MULT_BREGION/statespace_euclidian/{animal}-{date}-logistic-combine={combine}/statespace_euclidian/bregion={bregion}/ver={scalar_or_traj}-{version}-subtrmean={subtrmean}-projtwind={projtwind}-npcs={npcs}"
            os.makedirs(savedir, exist_ok=True)

            ### Project trials onto the axis (base1 - base2)
            X = PAredu.X.squeeze(axis=2).T # (trials, ndims)
            dflab = PAredu.Xlabels["trials"]

            if False:
                # - get mean activity for base1, base2
                inds = dflab[dflab["idx_morph_temp"] == 0].index.tolist()
                xmean_base1 = np.mean(X[inds,:], axis=0) # (ndims,)

                inds = dflab[dflab["idx_morph_temp"] == 99].index.tolist()
                xmean_base2 = np.mean(X[inds,:], axis=0) # (ndims,)

                # get mean projected data for each state
                Xproj, fig = projection_onto_axis_subspace(xmean_base1, xmean_base2, X, doplot=True)
                savefig(fig, f"{savedir}/morphset={morphset}-projections_preprocess.pdf")

            else:
                # Do this in cross-validated way -- split base prims into two sets, run this 2x, then 
                # average over base prims.
                from pythonlib.tools.statstools import crossval_folds_indices

                inds_pool_base1 = dflab[dflab["idx_morph_temp"] == 0].index.values
                inds_pool_base2 = dflab[dflab["idx_morph_temp"] == 99].index.values
                list_inds_1, list_inds_2 = crossval_folds_indices(len(inds_pool_base1), len(inds_pool_base2), 2)

                # shuffle inds
                np.random.shuffle(inds_pool_base1)
                np.random.shuffle(inds_pool_base2)

                list_inds_1 = [inds_pool_base1[_inds] for _inds in list_inds_1]
                list_inds_2 = [inds_pool_base2[_inds] for _inds in list_inds_2]

                list_X =[]
                for _i, (inds_base_1, inds_base_2) in enumerate(zip(list_inds_1, list_inds_2)):
                    # inds_base_1 = inds_pool_base1[_inds1]
                    # inds_base_2 = inds_pool_base2[_inds2]
                    # print(inds_base_1)

                    # - get mean activity for base1, base2
                    xmean_base1 = np.mean(X[inds_base_1,:], axis=0) # (ndims,)
                    xmean_base2 = np.mean(X[inds_base_2,:], axis=0) # (ndims,)

                    # get mean projected data for each state
                    xproj, fig = projection_onto_axis_subspace(xmean_base1, xmean_base2, X, doplot=True)
                    savefig(fig, f"{savedir}/morphset={morphset}-projections_preprocess-iter={_i}.pdf")

                    # replace the train data with nan
                    xproj[inds_base_1] = np.nan
                    xproj[inds_base_2] = np.nan

                    list_X.append(xproj)
                # Get average over iterations
                Xproj = np.nanmean(np.stack(list_X), axis=0)

            dfproj = PAredu.Xlabels["trials"].copy()
            dfproj["x_proj"] = Xproj

            if DOPLOTS:
                fig = sns.catplot(data=dfproj, x="idx_morph_temp", y="x_proj", jitter=True, alpha=0.5)
                for ax in fig.axes.flatten():
                    ax.axhline(0, color="k", alpha=0.5)
                    ax.axhline(1, color="k", alpha=0.5)
                savefig(fig, f"{savedir}/morphset={morphset}-catplot-1.pdf")

                fig = sns.catplot(data=dfproj, x="idx_morph_temp", y="x_proj", kind="point")
                for ax in fig.axes.flatten():
                    ax.axhline(0, color="k", alpha=0.5)
                    ax.axhline(1, color="k", alpha=0.5)
                savefig(fig, f"{savedir}/morphset={morphset}-catplot-2.pdf")

                fig = sns.catplot(data=dfproj, x="idx_morph_temp", y="x_proj", kind="violin")
                for ax in fig.axes.flatten():
                    ax.axhline(0, color="k", alpha=0.5)
                    ax.axhline(1, color="k", alpha=0.5)
                savefig(fig, f"{savedir}/morphset={morphset}-catplot-3.pdf")

            plt.close("all")


            ### COLLECT ALL
            # Rescale so that 0 <--> 99 maps onto 0 <--> 1 index, so can compare across expts
            idx_max = dfproj[dfproj["idx_morph_temp"] < 99]["idx_morph_temp"].max()

            def f(x):
                if x<=idx_max:
                    return x
                elif x>=99:
                    return x - 99 + idx_max + 1
                else:
                    assert False
                
            dfproj["idx_morph_rescaled"] = [f(x) for x in dfproj["idx_morph_temp"]]
            from pythonlib.tools.listtools import rank_items
            # assert np.all(dfproj["idx_morph_rescaled"].tolist() == list(rank_items(dfproj["idx_morph_temp"]-1, method="dense"))), "sanity cehck."
            dfproj["idx_morph_rescaled"] = dfproj["idx_morph_rescaled"]/(idx_max+1) # rescale
            # dfproj.groupby(["idx_morph_temp", "idx_morph_rescaled"]).size()

            dfproj["animal"] = animal
            dfproj["date"] = date
            dfproj["morphset"] = morphset
            dfproj["bregion"] = bregion

            list_dfproj.append(dfproj)

DFPROJ = pd.concat(list_dfproj).reset_index(drop=True)
DFPROJ = append_col_with_grp_index(DFPROJ, ["animal", "date", "morphset"], "ani_date_mrp")

/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240515-logistic-combine=True/statespace_euclidian/bregion=M1/morphset=0-ver=scal-dpca-subtrmean=False-projtwind=(0.1, 1.2)-npcs=8/Cldist.pkl
/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240515-logistic-combine=True/statespace_euclidian/bregion=M1/morphset=0-ver=scal-dpca-subtrmean=False-projtwind=(0.1, 1.2)-npcs=8/PAredu.pkl
/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240515-logistic-combine=True/statespace_euclidian/bregion=PMv/morphset=0-ver=scal-dpca-subtrmean=False-projtwind=(0.1, 1.2)-npcs=8/Cldist.pkl
/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240515-logistic-combine=True/statespace_euclidian/bregion=PMv/morphset=0-ver=scal-dpca-subtrmean=False-projtwind=(0.1, 1.2)-npcs=8/PAredu.pkl
/lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/Diego-240515-logistic-combine=True/statespace_euclidian/bregion=PMd/morphset=0-ver=scal-dpca-su

In [9]:
from pythonlib.tools.pandastools import append_col_with_grp_index
from pythonlib.tools.pandastools import plot_subplots_heatmap

DFPROJ = pd.concat(list_dfproj).reset_index(drop=True)
DFPROJ = append_col_with_grp_index(DFPROJ, ["animal", "date", "morphset"], "ani_date_mrp")

##### COMBINED PLOTS

In [10]:
SAVEDIR = f"{SAVEDIR_BASE}/MULT_BREGION/statespace_euclidian/COMBINED"
os.makedirs(SAVEDIR, exist_ok=True)

In [14]:

# Optionally remove flankers
for exclude_flankers in [False, True]:

    if exclude_flankers:
        DFPROJ_THIS = DFPROJ[(DFPROJ["idx_morph_temp"]>=0) & (DFPROJ["idx_morph_temp"]<=99)].reset_index(drop=True)
    else:
        DFPROJ_THIS = DFPROJ

    fig, axes = plot_subplots_heatmap(DFPROJ_THIS, "ani_date_mrp", "idx_morph_temp", "x_proj", "bregion", share_zlim=True, ZLIMS=[-0.2, 1.2])
    savefig(fig, f"{SAVEDIR}/heatmaps_all-idx_morph_temp-exclude_flankers={exclude_flankers}.pdf")

    fig, axes = plot_subplots_heatmap(DFPROJ_THIS, "ani_date_mrp", "idx_morph_rescaled", "x_proj", "bregion", share_zlim=True, ZLIMS=[-0.2, 1.2])
    savefig(fig, f"{SAVEDIR}/heatmaps_all-idx_morph_rescaled-exclude_flankers={exclude_flankers}.pdf")

    if False:
        # fig = sns.relplot(data=DFPROJ, x="idx_morph_rescaled", y="x_proj", hue="ani_date_mrp", col="bregion")
        fig = sns.relplot(data=DFPROJ_THIS, x="idx_morph_rescaled", y="x_proj", hue="ani_date_mrp", col="bregion", kind="line")
        for ax in fig.axes.flatten():
            ax.axhline(0)
            ax.axhline(1)
            # ax.set_ylim(0, 1)
        savefig(fig, f"{SAVEDIR}/heatmaps_all-idx_morph_rescaled-exclude_flankers={exclude_flankers}.pdf")

    if False:
        fig = sns.relplot(data=DFPROJ_THIS, x="idx_morph_rescaled", y="x_proj", col="bregion", kind="line")
        for ax in fig.axes.flatten():
            ax.axhline(0)
            ax.axhline(1)
    
plt.close("all")


/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pythonlib/pythonlib/tools/pandastools.py:1142: RuntimeWarning: Mean of empty slice
  valthis = np.nanmean(dfsub[val_name])
/home/lucas/code/pyt

##### Compoare logistic vs. linear fit

In [ ]:
# [Devo]

if True:
    dfthis = dfproj[(dfproj["idx_morph_temp"]>=0) & (dfproj["idx_morph_temp"]<=99)].reset_index(drop=True)
else:
    dfthis = dfproj
# Compare fit using logistic vs. linear curve

var_score = "x_proj"
var_x = "idx_morph_rescaled"
Y = dfthis[var_score].values
X = dfthis[var_x].values
X_cat = dfthis["idx_morph_temp"].tolist()
nfolds = 2
doplot = True

dfres_summary, dfres = curve_fit_and_test_wrapper_compare_logistic_linear(X, Y, X_cat, nfolds, doplot)


In [15]:
# [GOOD] pipeline
from pythonlib.tools.statstools import curve_fit_and_test
from pythonlib.tools.statstools import balanced_stratified_kfold, curve_fit_and_test_wrapper_compare_logistic_linear

from pythonlib.tools.pandastools import grouping_append_and_return_inner_items_good
grpdict = grouping_append_and_return_inner_items_good(DFPROJ, ["animal", "date", "morphset", "bregion"])

# list_bregion = ["PMv", "PMd", "M1"]
# list_bregion = ["PMv", "PMd", "SMA", "preSMA", "M1", "FP"]
list_bregion = _REGIONS_IN_ORDER_COMBINED

for exclude_flankers in [False, True]:

    savedir = f"{SAVEDIR}/logistic_vs_linear=exclude_flankers={exclude_flankers}"
    os.makedirs(savedir, exist_ok=True)
    print("... Saving figures at: ", savedir)

    RES = []
    for (animal, date, morphset, bregion), inds in grpdict.items():
        
        if bregion not in list_bregion:
            continue

        print((animal, date, morphset, bregion))
        dfproj = DFPROJ.iloc[inds].reset_index(drop=True)

        ##### Compare fit using logistic vs. linear curve
        # Optionally remove flankers
        if exclude_flankers:
            dfthis = dfproj[(dfproj["idx_morph_temp"]>=0) & (dfproj["idx_morph_temp"]<=99)].reset_index(drop=True)
        else:
            dfthis = dfproj

        # Compare fit using logistic vs. linear curve
        var_score = "x_proj"
        var_x = "idx_morph_rescaled"
        Y = dfthis[var_score].values
        X = dfthis[var_x].values
        X_cat = dfthis["idx_morph_temp"].tolist()
        nfolds = 8
        doplot = False

        dfres_summary, dfres = curve_fit_and_test_wrapper_compare_logistic_linear(X, Y, X_cat, nfolds, doplot)

        if dfres_summary is None:
            print("Failed to converge for: ", bregion)
            continue

        dfres_summary["animal"] = animal
        dfres_summary["date"] = date
        dfres_summary["morphset"] = morphset
        dfres_summary["bregion"] = bregion

        RES.append(dfres_summary)

    DFRES = pd.concat(RES).reset_index(drop=True)


    from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
    DFRES = append_col_with_grp_index(DFRES, ["date", "morphset"], "date_morphset")
    DFRES = append_col_with_grp_index(DFRES, ["animal", "date", "morphset"], "animal_date_morphset")


    from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
    # import seaborn as sns
    # sns.catplot(data=DFRES, x="model", y="R2", col="bregion", kind="point")
    # sns.catplot(data=DFRES, x="model", y="R2_mean", col="bregion", kind="point")
    # sns.catplot(data=DFRES, x="model", y="R2", col="bregion", kind="point")
    for animal in DFRES["animal"].unique():
        dfres = DFRES[DFRES["animal"] == animal].reset_index(drop=True)
        for var_score in ["R2"]:
            for plot_text in [False, True]:
                _, fig = plot_45scatter_means_flexible_grouping(dfres, "model", "linear", "sigmoid", "bregion", 
                                                                var_score, "date_morphset", shareaxes=True, SIZE=3, fontsize=5,
                                                                plot_text=plot_text)
                savefig(fig, f"{savedir}/scatter-animal={animal}-varscore={var_score}-text={plot_text}-1.pdf")

                _, fig = plot_45scatter_means_flexible_grouping(dfres, "model", "linear", "sigmoid", "animal_date_morphset", 
                                                                var_score, "bregion", shareaxes=True, SIZE=3, fontsize=5,
                                                                plot_text=plot_text)
                savefig(fig, f"{savedir}/scatter-animal={animal}-varscore={var_score}-text={plot_text}-2.pdf")

                _, fig = plot_45scatter_means_flexible_grouping(dfres, "model", "linear", "sigmoid", "animal", 
                                                                            var_score, "bregion", shareaxes=True, SIZE=3, fontsize=5,
                                                                            plot_text=plot_text)
                savefig(fig, f"{savedir}/scatter-animal={animal}-varscore={var_score}-text={plot_text}-3.pdf")

                plt.close("all")

    dftmp, fig = plot_45scatter_means_flexible_grouping(DFRES, "model", "linear", "sigmoid", "animal", 
                                                                var_score, "bregion", shareaxes=True, SIZE=3, fontsize=5,
                                                                plot_text=plot_text)
    savefig(fig, f"{savedir}/scatter-by_animal.pdf")
    
    from pythonlib.tools.pandastools import summarize_featurediff
    dfsummary, dfsummaryflat, COLNAMES_NOABS, COLNAMES_ABS, COLNAMES_DIFF = summarize_featurediff(DFRES, 
                                                                                        "model", 
                                                                                        ["linear", "sigmoid"], ["R2"], 
                                                                                        ["animal", "date", "morphset", "bregion", "date_morphset", "animal_date_morphset"])

    # Directly compare two areas
    var_score = "R2"
    dftmp, fig = plot_45scatter_means_flexible_grouping(dfsummaryflat, "bregion", "PMv", "PMd", "animal", 
                                                                "value", "date_morphset", shareaxes=True, SIZE=3, fontsize=5,
                                                                plot_text=plot_text)
    savefig(fig, f"{savedir}/scatter-compare-PMd-vs-PMv.pdf")


    fig = sns.catplot(data=dfsummaryflat, x="bregion", y="value", hue="date_morphset", col="animal", legend=False)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/catplot-1.pdf")

    fig = sns.catplot(data=dfsummaryflat, x="bregion", y="value", col="animal", legend=False, kind="bar")
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.5)
    savefig(fig, f"{savedir}/catplot-2.pdf")

    plt.close("all")


... Saving figures at:  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/MULT_BREGION/statespace_euclidian/COMBINED/logistic_vs_linear=exclude_flankers=False
('Diego', 240515, 0, 'FP')
('Diego', 240517, 0, 'FP')
('Diego', 240731, 0, 'FP')
('Diego', 240801, 0, 'FP')
('Diego', 240802, 0, 'FP')
('Diego', 240515, 0, 'M1')
('Diego', 240517, 0, 'M1')
('Diego', 240731, 0, 'M1')
('Diego', 240801, 0, 'M1')
('Diego', 240802, 0, 'M1')
('Diego', 240515, 0, 'PMd')
('Diego', 240517, 0, 'PMd')
('Diego', 240731, 0, 'PMd')
('Diego', 240801, 0, 'PMd')
('Diego', 240802, 0, 'PMd')
('Diego', 240515, 0, 'PMv')
('Diego', 240517, 0, 'PMv')
('Diego', 240731, 0, 'PMv')
('Diego', 240801, 0, 'PMv')
('Diego', 240802, 0, 'PMv')
('Diego', 240515, 0, 'SMA')
('Diego', 240517, 0, 'SMA')
('Diego', 240731, 0, 'SMA')
('Diego', 240801, 0, 'SMA')
('Diego', 240802, 0, 'SMA')
('Diego', 240515, 0, 'dlPFC')
('Diego', 240517, 0, 'dlPFC')
('Diego', 240731, 0, 'dlPFC')
('Diego', 240801, 0, 'dlPFC')
('Diego', 240802, 

/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof

... Saving figures at:  /lemur2/lucas/analyses/recordings/main/decode_moment/PSYCHO_SP/MULT_BREGION/statespace_euclidian/COMBINED/logistic_vs_linear=exclude_flankers=True
('Diego', 240515, 0, 'FP')
('Diego', 240517, 0, 'FP')
('Diego', 240731, 0, 'FP')
Failed to converge for:  FP
('Diego', 240801, 0, 'FP')
('Diego', 240802, 0, 'FP')
('Diego', 240515, 0, 'M1')
('Diego', 240517, 0, 'M1')
('Diego', 240731, 0, 'M1')
('Diego', 240801, 0, 'M1')
('Diego', 240802, 0, 'M1')
('Diego', 240515, 0, 'PMd')
('Diego', 240517, 0, 'PMd')
('Diego', 240731, 0, 'PMd')
('Diego', 240801, 0, 'PMd')
('Diego', 240802, 0, 'PMd')
('Diego', 240515, 0, 'PMv')
('Diego', 240517, 0, 'PMv')
('Diego', 240731, 0, 'PMv')
('Diego', 240801, 0, 'PMv')
('Diego', 240802, 0, 'PMv')
('Diego', 240515, 0, 'SMA')
('Diego', 240517, 0, 'SMA')
('Diego', 240731, 0, 'SMA')
('Diego', 240801, 0, 'SMA')
('Diego', 240802, 0, 'SMA')
('Diego', 240515, 0, 'dlPFC')
('Diego', 240517, 0, 'dlPFC')
('Diego', 240731, 0, 'dlPFC')
('Diego', 240801, 0, 

/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/lucas/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof

In [29]:
type(dflab["aborted"].tolist()[0])

bool

In [ ]:
dflab["aboe"]

In [34]:
from pythonlib.tools.pandastools import aggregGeneral
aggregGeneral(dflab, ["seqc_0_shape", "seqc_0_loc"], ["seqc_0_locy"], "all")

1 ['shape_this_event', 'seqc_3_shapesemcat', 'seqc_2_locon_binned', 'seqc_5_shapesemcat', 'character', 'seqc_4_loc', 'seqc_4_shapesemgrp', 'seqc_3_locon', 'seqc_5_shapesem', 'seqc_5_locon_binned', 'seqc_2_shapesemcat', 'seqc_1_shapesem', 'seqc_7_shapesem', 'seqc_6_shapesem', 'seqc_7_loc', 'seqc_6_loc_on_clust', 'seqc_0_loc_on_clust', 'seqc_5_loc_on_clust', 'seqc_3_loc', 'seqc_6_shape', 'seqc_1_locon_binned', 'seqc_2_angle_binned', 'seqc_3_shapesemgrp', 'seqc_1_center_binned', 'seqc_5_locon', 'seqc_6_locon_binned', 'seqc_7_center_binned', 'seqc_7_shapesemcat', 'loc_this_event', 'seqc_5_shape', 'seqc_2_shapesem', 'seqc_4_center_binned', 'shape_is_novel_list', 'seqc_0_locon_binned', 'seqc_2_loc', 'seqc_7_locon_binned', 'seqc_1_loc_on_clust', 'seqc_5_center_binned', 'taskconfig_shploc_SHSEM', 'seqc_6_angle_binned', 'seqc_4_shapesem', 'seqc_5_loc', 'seqc_7_shapesemgrp', 'seqc_7_loc_on_clust', 'taskconfig_shploc', 'seqc_2_loc_on_clust', 'seqc_6_shapesemcat', 'seqc_6_loc', 'taskconfig_shp', '

,seqc_0_shape,seqc_0_loc,seqc_0_locy,shape_this_event,seqc_3_shapesemcat,seqc_2_locon_binned,seqc_5_shapesemcat,seqc_4_loc,seqc_4_shapesemgrp,seqc_3_locon,...,seqc_3_angle_binned,seqc_7_shape,seqc_3_center_binned,seqc_0_shapesem,which_level,event,bregion,twind,shapes_drawn,locs_drawn
0,line-10-10.0-10.0,"(0, 1)",177.777081,line-10-10.0-10.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-10-10.0-10.0,)","((0, 1),)"
1,line-10-10.0-10.0,"(1, 0)",24.654790,line-10-10.0-10.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-10-10.0-10.0,)","((1, 0),)"
2,line-13-13.0-13.0,"(0, 1)",172.104376,line-13-13.0-13.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-13-13.0-13.0,)","((0, 1),)"
3,line-13-13.0-13.0,"(1, 0)",11.588747,line-13-13.0-13.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-13-13.0-13.0,)","((1, 0),)"
4,line-18-18.0-18.0,"(0, 1)",295.229094,line-18-18.0-18.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-18-18.0-18.0,)","((0, 1),)"
5,line-18-18.0-18.0,"(1, 0)",136.401290,line-18-18.0-18.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-18-18.0-18.0,)","((1, 0),)"
6,line-20-20.0-20.0,"(0, 1)",183.207727,line-20-20.0-20.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-20-20.0-20.0,)","((0, 1),)"
7,line-20-20.0-20.0,"(1, 0)",25.910987,line-20-20.0-20.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-20-20.0-20.0,)","((1, 0),)"
8,line-21-21.0-21.0,"(0, 1)",177.403029,line-21-21.0-21.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-21-21.0-21.0,)","((0, 1),)"
9,line-21-21.0-21.0,"(1, 0)",22.069063,line-21-21.0-21.0,IGN,"(IGN, IGN)",IGN,"(IGN, IGN)",IGN,"(nan, nan)",...,IGN,IGN,"(IGN, IGN)",line-UU-UU,trial,03_samp,preSMA,"(-1.0, 1.8)","(line-21-21.0-21.0,)","((1, 0),)"


In [32]:
dflab

KeyError: 'morph_set_idx'

# OLDER STUFF

### Plot single trials

In [ ]:
# TRAINING - Plot an example training trial

indtrial = 13
dflab = PAtrain.Xlabels['trials']
print(f"For this trial {indtrial}, {Dc.VarDecode} = {dflab.iloc[indtrial][Dc.VarDecode]}")
Dc.plot_single_trial(indtrial, PA=PAtrain);


In [ ]:
# TESTING

In [ ]:
# First, extract PA

idx_in_morphset = 4
trialcodes = find_morphset_morphidx(DSmorphsets, morphset, idx_in_morphset)


[(i, " --- ", map_tcmorphset_to_info[(tc, morphset)]) for i, tc in enumerate(trialcodes)]

In [ ]:
#

tc = trialcodes[13]

dflab = PAtest.Xlabels["trials"]

indtrial = dflab[dflab["trialcode"] == tc].index.tolist()[0]

print("Trialcode: ", tc, ", idx=", indtrial)
print(f"For this trial {indtrial}, {Dc.VarDecode} = {dflab.iloc[indtrial][Dc.VarDecode]}")

# What was actually drawn on this trial?
print("What was actually drawn on this trial?", map_tcmorphset_to_info[(tc, morphset)])

tbin_dur=0.15
tbin_slide=0.01
if False:
    Dc.plot_single_trial(indtrial, PA=PAtest, tbin_dur=tbin_dur, tbin_slide=tbin_slide);
else:
    # Hack, testing out if including null helps
    Dc.plot_single_trial(indtrial, PA=PAtest, tbin_dur=tbin_dur, tbin_slide=tbin_slide);
    DcNoNull.plot_single_trial(indtrial, PA=PAtest, tbin_dur=tbin_dur, tbin_slide=tbin_slide);
    DcNull.plot_single_trial(indtrial, PA=PAtest, tbin_dur=tbin_dur, tbin_slide=tbin_slide);


In [ ]:
# TODO

# Timecourse plots

### Useing single case for the one trinaed on [B1 M B2]

##### A single decode train-test split (i.e., is not good, not testing on held out)

In [ ]:
dfscores_testsplit, dfscores_usertest, decoders, trainsets, PAtest

In [ ]:
idx_trainsplit = 0
Dc = decoders[idx_trainsplit]
PAtrain_orig = trainsets[idx_trainsplit]["PAtrain_orig"]
test_index_PAtrain_orig = trainsets[idx_trainsplit]["test_index_PAtrain_orig"]

# Slice out the test dataset
if False:
    patest = PAtrain_orig.slice_by_dim_indices_wrapper("trials", test_index_PAtrain_orig, reset_trial_indices=True)
else:
    patest = PAtrain_orig


##### [Good] loop thru each train-test split, and collect data from test trials.

In [ ]:
# For the same decoder idx, overlay means from diff trials

assert False, "in progress -- need to color each line differntly.."

decoder_idx = 0
list_inds = [[0,1,2], [3,4,5]]

fig, ax = plt.subplots()
for indstrials in list_inds:

    probs_mat_all_this = probs_mat_all[indstrials, :, decoder_idx][:, :, None]
    Dc._timeseries_plot_by_shape_drawn_order(probs_mat_all_this, times, ["test"], MAP_INDEX_TO_COL, ax)
    # ax.set_title(f"{idx_within}, {assigned}")
    # ax.set_title(f"{idx_within}, {_assigned}")


In [ ]:
dfres = pd.DataFrame(res)
dfres

In [ ]:
# Q: is (base1 decode during ambig_base2) higher than (base1 decode during base2)

decoder_class = 99




def _extract_probs_mat(dfres, labels, idx_within, _assigned, decoder_class):
    tmp = dfres[(dfres["idx_within"] == idx_within) & (dfres["_assigned"] == _assigned)]
    assert len(tmp)==1
    probs_mat_all_this = tmp["probs_mat_all_this"].values[0] # (trials, times, labels)
    idx_decode = labels.index(decoder_class)
    probs_mat = probs_mat_all_this[:, :, idx_decode]
    return probs_mat

idx_within = 3
_assigned = "ambig_base1"
probs_mat_1 = _extract_probs_mat(dfres, labels, idx_within, _assigned, decoder_class)

idx_within = 0
_assigned = "base1"
probs_mat_2 = _extract_probs_mat(dfres, labels, idx_within, _assigned, decoder_class)

probs_vec_1 = np.mean(probs_mat_1, axis=0)
probs_vec_2 = np.mean(probs_mat_2, axis=0)

# take diff
probs_vec_diff = probs_vec_1 - probs_vec_2

fig, ax = plt.subplots()

ax.plot(times, probs_vec_diff)


In [ ]:
# Subtract baseline trials (when the)

### METHOD -- trial by trial scatter plots [including KDE plots...]

In [ ]:
if False:
    indtrials = list(range(20))
elif False:
    indtrials = PAtest.Xlabels["trials"][PAtest.Xlabels["trials"][Dc.VarDecode]=="0|99"].index
else:
    # First, extract PA
    morphset = 3
    idx_in_morphset = 2
    trialcodes = find_morphset_morphidx(DSmorphsets, morphset, idx_in_morphset)
    display([(i, " --- ", map_tcmorphset_to_info[(tc, morphset)]) for i, tc in enumerate(trialcodes)])

    indtrials = PAtest.Xlabels["trials"][PAtest.Xlabels["trials"]["trialcode"].isin(trialcodes)].index.tolist()
    indtrials

print(len(indtrials))
twind = (0.1, 1.8)
PAprobs, probs_mat_all, times, labels = Dc.timeseries_score_wrapper(PAtest, twind, indtrials)

In [ ]:
import seaborn as sns

In [ ]:
fig, ax = plt.subplots()
x1 = probs_mat_all[0, :, 0]
x2 = probs_mat_all[0, :, 1]

def _plot_pairwise(xvals, yvals, ax):

    # ax.plot(x1, x2, "-x", alpha=0.5)
    # sns.scatterplot(x=xvals, y=yvals, ax=ax, color="k", alpha=0.02)
    
    sns.kdeplot(x=xvals, y=yvals, ax=ax, fill=True, thresh=0, levels=50, cmap="mako")
    
    # This is the worst
    # sns.histplot(x=xvals, y=yvals, ax=ax, bins=20, stat="probability", cbar=True, alpha=0.5)
    # sns.rugplot(x=xvals, y=yvals, ax=ax, color="k", alpha=0.5)

In [ ]:
fig, ax = plt.subplots()
x1 = probs_mat_all[0, :, 0]
x2 = probs_mat_all[0, :, 1]
_plot_pairwise(x1, x2, ax)

In [ ]:
# Make plot, comparing each pair of labels

ncols = len(labels)
nrows = len(labels)
SIZE = 3
fig, axes = plt.subplots(nrows, ncols, figsize=(nrows*SIZE, ncols*SIZE))

for i, lab1 in enumerate(labels): # rows
    for j, lab2 in enumerate(labels): # cols
        if j!=i:
            ax = axes[i][j]

            xvals = probs_mat[:, j]
            yvals = probs_mat[:, i]
            # ax.set_title((i, j))
            # ax.set_title((i, j))
            _plot_pairwise(xvals, yvals, ax)
            ax.set_xlabel(lab2)
            ax.set_ylabel(lab1)

            # assert False
